## EDA And Sensor Selection

### Work flow

1. Read in sensor and pull associated network
2. Graphical analysis of system
3. Get sensor data and analyse for completeness etc
4. Select sensors for study
5. Output OSM network of road network and associated sensors



In [80]:
# Import modules

import pandas as pd
import osmnx as ox
import numpy as np
import shapely
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import random
import mysql.connector

In [2]:
# Get sensors
sensors = pd.read_csv('Data/midas_sensor_locations.csv')

#Get network for area just for pimary roads etc
cf = '["highway"~"motorway|motorway_link|primary|trunk"]'
G = ox.graph_from_place('Kent, United Kingdom', network_type='drive', custom_filter=cf, simplify=False)
edge_attributes = ox.graph_to_gdfs(G, nodes=True)[1]

#Get edge centroids
edge_attributes['cent_x'] = edge_attributes['geometry'].centroid.x
edge_attributes['cent_y'] = edge_attributes['geometry'].centroid.y
#Add edge index
edge_attributes['edge_index'] = range(len(edge_attributes))
#Get boundary
graph_boundary=shapely.geometry.box(edge_attributes.geometry.total_bounds[0],edge_attributes.geometry.total_bounds[1],edge_attributes.geometry.total_bounds[2],edge_attributes.geometry.total_bounds[3])

c:\Users\cconlan\AppData\Local\anaconda3\envs\short\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  # Remove the CWD from sys.path while we load stuff.
c:\Users\cconlan\AppData\Local\anaconda3\envs\short\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  # This is added back by InteractiveShellApp.init_path()


In [69]:
#Output network, sensor and boundary to view in qgis

edge_attributes_gdf = GeoDataFrame(edge_attributes, crs="EPSG:4326", geometry=edge_attributes['geometry'])
edge_attributes_gdf.to_file("Data/examples/kent_road_network.json", driver="GeoJSON")

sensor_points = []
for i,r in sensors.iterrows():
    sensor_points.append(Point([r['Longitude'],r['Latitude']]))
sensors['geometry'] = sensor_points

sensors_in_study = sensors[[graph_boundary.contains(point) for point in sensor_points]]

sensors_gdf = GeoDataFrame(sensors_in_study, crs="EPSG:4326", geometry=sensors_in_study['geometry'])
sensors_gdf.to_file("Data/examples/sensor_locations.json", driver="GeoJSON")

c:\Users\cconlan\AppData\Local\anaconda3\envs\short\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [70]:
sensor_nearest_edges = ox.distance.nearest_edges(G, list(sensors_in_study['Longitude']), list(sensors_in_study['Latitude']), interpolate=None, return_dist=False)

c:\Users\cconlan\AppData\Local\anaconda3\envs\short\lib\site-packages\osmnx\distance.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dists = geoms.iloc[list(rtree.nearest(xy))].distance(Point(xy))
c:\Users\cconlan\AppData\Local\anaconda3\envs\short\lib\site-packages\osmnx\distance.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dists = geoms.iloc[list(rtree.nearest(xy))].distance(Point(xy))
c:\Users\cconlan\AppData\Local\anaconda3\envs\short\lib\site-packages\osmnx\distance.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dists = geoms.iloc[list(rtree.ne

In [78]:
#Randomly sample sensors to output sensor and nearest edge to check

rand_sens = random.randrange(0,len(sensors_in_study)-1)

sensor_to_check = sensors_gdf.iloc[rand_sens].to_frame().T
edge_to_check = edge_attributes.loc[sensor_nearest_edges[rand_sens]].to_frame().T#

sensor_check_gdf = GeoDataFrame(sensor_to_check, crs="EPSG:4326", geometry=sensor_to_check['geometry'])
sensor_check_gdf['geometry'].to_file("Data/examples/sensor_check.json", driver="GeoJSON")

edge_check_gdf = GeoDataFrame(edge_to_check, crs="EPSG:4326", geometry=edge_to_check['geometry'])
edge_check_gdf['geometry'].to_file("Data/examples/edge_check.json", driver="GeoJSON")

## Time Series Exploratory Data Analysis

- Completeness
- Methods for missing data imputation
- Analyse stationarity of data
- Analyse any seasonanality

In [ ]:
dec_to_min_dict = {
    0.0 : 0,
    0.25 : 15,
    0.50 : 30,
    0.75 : 45
    }

In [82]:
# Replace these placeholders with your actual database credentials
host = "localhost"
user = "root "
password = "NcFc2009S"
database = "midas"

# Establish a database connection
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = connection.cursor(buffered=True)

In [84]:
roads = []

for i in list(sensors_in_study['site_ID']):
    roads.append(i.split('/')[0])

sensors_in_study['Road'] = roads

M20     46
M25     24
M2      18
A2       2
5823     1
5845     1
5889     1
5892     1
5993     1
9949     1
A20      1
Name: Road, dtype: int64


c:\Users\cconlan\AppData\Local\anaconda3\envs\short\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [147]:
rand_sens = random.randrange(0,len(sensors_in_study)-1)
next_sensor = sensors_gdf.iloc[rand_sens].to_frame().T
print(next_sensor)

      site_ID Latitude Longitude count(*)                  geometry Road
62  M20/6576A  51.2955  0.528171   277002  POINT (0.528171 51.2955)  M20


In [148]:
# For each sensor get a measure of completeness

#For each sensor get time-series data from 2022

sql_query = "select * from full_data where site_ID = '{}' and yr = '2022'".format(next_sensor['site_ID'].values[0])
cursor.execute(sql_query)
result = cursor.fetchall()
ts = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])

# Add date/time index

i, d = divmod((ts['time_period'] / 4), 1)
ts['Minute'] = [*map(dec_to_min_dict.get, list(d))]

years = []
months = []
days = []

for i,r in ts.iterrows():
    years.append(r['data_date'].year)
    months.append(r['data_date'].month)
    days.append(r['data_date'].day)

ts['Year'] = years
ts['Month'] = months
ts['Day'] = days
ts['Hour'] = ts['data_hour']

ts['Datetime'] = pd.to_datetime(ts[['Year', 'Month', 'Day', 'Hour', 'Minute']])
ts.set_index('Datetime', inplace=True)
ts = ts.sort_index()

#ts = ts[['Lane', 'Short_Veh', 'Long_Veh', 'Total_Flow']]

In [149]:
# Count zeros
zero_count_s = (ts['Short_Veh'] == 0).sum()
print(zero_count_s)

zero_count_l = (ts['Long_Veh'] == 0).sum()
print(zero_count_l)

4673
14779


In [150]:
# Count nulls
null_count_s = ts['Short_Veh'].isnull().sum()
print(null_count_s)

null_count_l = ts['Long_Veh'].isnull().sum()
print(null_count_l)

0
0


In [151]:
# Identify and count outliers (assuming values more than 2 standard deviations from the mean are outliers)
mean_value = ts['Short_Veh'].mean()
std_dev = ts['Short_Veh'].std()
outlier_count_s = ((ts['Short_Veh'] > mean_value + 2 * std_dev) | (ts['Short_Veh'] < mean_value - 2 * std_dev)).sum()
print(outlier_count_s)

mean_value = ts['Long_Veh'].mean()
std_dev = ts['Long_Veh'].std()
outlier_count_l = ((ts['Long_Veh'] > mean_value + 2 * std_dev) | (ts['Long_Veh'] < mean_value - 2 * std_dev)).sum()
print(outlier_count_l)

2411
5449
